# 1 Last week we used the activation function. Why is this not used with backpropagation?

__A:__ The function is not differentiable.

# 2 What is the minimum number of hidden neuron layers needed in order to approximate an arbitrary continuous function, and why?

__A:__ One hidden layer is the minimum. "Universal approximation theorem: Any continuous function
can be approximated by a neural network with a single
hidden layer", slide 54 https://www.uio.no/studier/emner/matnat/ifi/INF3490/h18/timeplan/slides/lecture6-1pp.pdf


Without a hidden layer, it is impossible to approximate non-linear separable problems. The first layer gives lines, and the second layer opens up the dimension to two, so e.g. traingles can be made. Furthermore, a compbination of enough traingles can approximate any shape. See slide 12 https://www.uio.no/studier/emner/matnat/ifi/INF3490/h18/timeplan/slides/lecture6-1pp.pdf

<mark> UNCLEAR <br>
    
    
    
# 3 Why do we use a validation set? Describe how the three different cross-validation methods presented in the lecture slides work, and what their advantages and disadvantages are.
We want the model to generalize well, meaning that it works well on other data than the data the model was estimated on. When fitting the model to the full dataset, the model will typically not fit well for other data. In oredr to create a model that generalizes well, the data is split into a training and a validation set. The model is fit to the training set, and is then evaluated on the validation set. The model performance on the validation set describes how well the model generalizes. <br>

Three crossvalidation methods. <br> 
1) Split into training and validation (and possible test). Works as the previous paragraph. <br>
2) K-fold cross-validation. Split into K number of folds. Each fold performs as validation set once, while at the same time the model is fit to the remaining folds. <br>
3) Bootstrapping. Draw random observations with replacement and use the rest as validation. Repeat.

# 4 Implement the MLP shown below, and train it to correctly perform the XOR function.

In [36]:
import numpy as np
import matplotlib.pyplot as plt
class NN:
    """ 2 layer (1 hidden). Single input combination of 2 intergers. Output 2 integers"""
    
    def __init__(self, nodeNumbers, activationFunction, learningRate, targetMatrix, inputMatrix, errorTolerance, \
                 maxIterations, test=False):
        self.nodeNumbers, self.activationFunction, self.learningRate, self.targetMatrix, self.inputMatrix, self.errorTolerance,  \
        self.maxIterations, self.test= nodeNumbers, activationFunction, learningRate, targetMatrix, inputMatrix, errorTolerance,\
        maxIterations, test
        
        
        self.weightMatrices = []
        
        # If single input
        if self.inputMatrix.ndim == 1:
            self.inputs = self.inputMatrix.tolist()
            self.targets = self.targetMatrix.tolist()
            self.targetLength = self.targets
        else:
            self.numberOfInputs = np.shape(self.inputMatrix)[1]
        
        if not test:
            for nodeNumber in range(len(nodeNumbers)-1):
                self.weightMatrices.append(np.random.random_sample((nodeNumbers[nodeNumber+1], \
                                                                    nodeNumbers[nodeNumber+1] +1)) - .5)          
        else:
            self.weightMatrices.append(np.array(((1., -1., 0.), (1., 0., 1.))))
            self.weightMatrices.append(np.array(((1., 1., 0.), (1., -1., 1.)))) 
            self.inputs = np.array((1, 0, 1))
            
    def run(self):
        self.forward()
        self.calculateErrors()
        self.iterations = 1
        self.outputForPlot1 = []
        self.outputForPlot2 = []
        while np.abs(self.error2) > self.errorTolerance and self.iterations < self.maxIterations: 
            self.backward()
            self.forward()
            #print('Output: ', self.outputs)
            self.calculateErrors()
            self.iterations += 1
            self.outputForPlot1.append(self.outputs[-1][0])
            self.outputForPlot2.append(self.outputs[-1][1])
        #self.plot()

        print('Iterations: ', self.iterations, '|Error|: ', self.error2, 'Output: ', self.outputs[1])
        
    def runMultipleInput(self):
            self.iterationNumber = 0
            numberOfAdjustmentsDuringIteration = 1
            
            while (self.iterationNumber < self.maxIterations and numberOfAdjustmentsDuringIteration != 0):
                self.iterationNumber += 1
                numberOfAdjustmentsDuringIteration = 0
                
                #print('targetMatrix', self.targetMatrix)

                for inputNumber in range(self.numberOfInputs):
                    #print('inputNumber', inputNumber)
                    self.inputs = self.inputMatrix[:,inputNumber]
                    self.targets = self.targetMatrix[inputNumber]
                    self.targetLength = np.asarray([0])
                    # FIX to allow for multidim output
                    #print('self.targetMatrix[inputNumber] ', self.targetMatrix[inputNumber] )
                    self.forward()
                    self.calculateErrors()
                    
                    #print('self.error2', self.error2)
                    if abs(self.error2[-1]) > self.errorTolerance:
                        numberOfAdjustmentsDuringIteration +=1
                        self.backward()
                        
            print('Iterations: ', self.iterationNumber, '|Error|: ', self.error2[-1])
        
    def plot(self):
        fig, (ax, ax2) = plt.subplots(1,2)
        ax.plot(np.arange(len(self.outputForPlot1)), self.outputForPlot1)#, label='Input 1')
        ax2.plot(np.arange(len(self.outputForPlot2)), self.outputForPlot2)#, label='Input 2')
        ax.set_title('Input 1')
        ax2.set_title('Input 22')
        plt.savefig('simple.pdf')
        
    def differentiateActivationFunction(self, weightedInputs):
        if isinstance(weightedInputs, np.ndarray):
            onesArray = np.ones(len(weightedInputs))
        else:
            onesArray = 1
        out = self.activationFunction(weightedInputs)*(onesArray - self.activationFunction(weightedInputs))
        #return 1
        #return out      
        
    
    def forward(self):
        weightedInputs = []
        print('self.inputs', self.inputs)
        weightedInputs.append(self.weightMatrices[0] @ self.inputs)
        self.outputs = []
        self.outputs.append(self.activationFunction(weightedInputs[-1]))

        for layer in range(1, len(self.nodeNumbers)-1):
            outputsIncludingBias = np.concatenate([[1], self.outputs[-1]])
            weightedInputs.append(self.weightMatrices[layer] @ outputsIncludingBias)
            self.outputs.append(self.activationFunction(weightedInputs[-1]))
        self.weightedInputs = weightedInputs
        #print('self.outputs', self.outputs[-1])
            
    def calculateErrors(self):
        self.error2 = 0
        print('type(self.targets)', type(self.targets))
        if not isinstance(self.targets, list):
                self.error2 += (self.targets - self.outputs[-1])**2
                #print('WRONG')
        else:
            for outputNodeNumber in range(len(self.outputs)):
            #print('outputNodeNumber', outputNodeNumber)
                self.error2 += (self.targets[outputNodeNumber] - \
                           self.outputs[-1][outputNodeNumber])**2
        
    def backward(self):
        # Output deltas
        self.deltaVectors = []
        '''
        deltaValuesOld = np.array([(self.outputs[-1][outputNumber] - self.targets[outputNumber])\
                      *self.differentiateActivationFunction(self.weightedInputs[-1][outputNumber]) \
                      for outputNumber in range(len(self.targets))])
        '''
        deltaValues = (self.outputs[-1] - self.targets)* self.differentiateActivationFunction(self.weightedInputs[-1])
        self.deltaVectors.append(deltaValues)
        
        # Hidden deltas
        if not isinstance(self.targets, np.ndarray):
            targetForLoop = np.array([self.targets])
        for layerNumber in range(len(self.nodeNumbers)-2):
            deltaVecs = []
            for j in range(self.nodeNumbers[-1-(layerNumber+1)+1]):
                deltaSum = 0
                #print('self.targetLength',self.targetLength)
                for k in range(len(self.targetLength)):
                    deltaSum += self.deltaVectors[-1-layerNumber][k] * self.weightMatrices[-1][k,j+1] 
                deltaVecs.append(deltaSum*self.differentiateActivationFunction(self.weightedInputs[-1][j]))
        self.deltaVectors.insert(0, deltaVecs)
        
        # Output weights
        outputsIncludingBias = np.concatenate([[1], self.outputs[-2]])
        for j in range(self.nodeNumbers[-2] + 1):
            #print(self.targetLength)
            for k in range(len(self.targetLength)):
                self.weightMatrices[-1][k, j] -= \
                self.learningRate*self.deltaVectors[1][k]*outputsIncludingBias[j]
        
        # Hidden weights
        for inputNumber in range(self.nodeNumbers[0]+1):
            for hiddenNodeNumber in range(self.nodeNumbers[1]): # fix indexes
                self.weightMatrices[0][hiddenNodeNumber, inputNumber] -= self.learningRate \
                * self.deltaVectors[0][hiddenNodeNumber]*self.inputs[inputNumber]
     
#def activationFunction(x):
 #   return x
 

def activationFunction(x):
    return 1./(1+np.exp(-x))
 

numberOfNodes = [2,2,2]
activationFunction = activationFunction
learningRate = 0.01
targetMatrix = np.array(((0), (1), (1), (0))).T
inputMatrix = np.array(((-1,0, 0), (-1, 0, 1), (-1,1, 0), (-1,1,1))).T
errorTolerance = 1e-3
maxIterations= 10000
test = False

nn=NN(numberOfNodes, activationFunction, learningRate, targetMatrix, inputMatrix, errorTolerance, maxIterations, test)
#nn.runMultipleInput()
'''
print('nn.weightMatrices',nn.weightMatrices)

nn.forward()
print('nn.weightedInputs',nn.weightedInputs)
print('nn.outputs',nn.outputs)

nn.calculateErrors()
print('nn.error2',nn.error2)

nn.backward()
print('nn.deltaVectors',nn.deltaVectors)
print('weightMatrices', nn.weightMatrices)

nn.run()
'''

"\nprint('nn.weightMatrices',nn.weightMatrices)\n\nnn.forward()\nprint('nn.weightedInputs',nn.weightedInputs)\nprint('nn.outputs',nn.outputs)\n\nnn.calculateErrors()\nprint('nn.error2',nn.error2)\n\nnn.backward()\nprint('nn.deltaVectors',nn.deltaVectors)\nprint('weightMatrices', nn.weightMatrices)\n\nnn.run()\n"

In [46]:
## Slides. Must change derivative inside class to 1
def activationFunction(x):
    return x

numberOfNodes,  activationFunction, learningRate, targetMatrix, inputMatrix, errorTolerance, maxIterations = \
[2,2,2], activationFunction, 0.1, np.array([1, 0]), np.array([0, 1]), 1e-6, 100
test = True


nn2=NN(numberOfNodes, activationFunction, learningRate, targetMatrix, inputMatrix, errorTolerance, maxIterations, test=True)


print('nn2.inputs',nn2.inputs)

print('nn.weightMatrices',nn2.weightMatrices)

nn2.forward()

print('nn.weightedInputs',nn2.weightedInputs)
print('nn.outputs',nn2.outputs)

nn2.calculateErrors()
print('nn.error2',nn2.error2)

nn2.backward()
#print('nn.deltaVectors',nn2.deltaVectors)
#print('weightMatrices', nn2.weightMatrices)
'''
#nn2.run()
'''

nn2.inputs [1 0 1]
nn.weightMatrices [array([[ 1., -1.,  0.],
       [ 1.,  0.,  1.]]), array([[ 1.,  1.,  0.],
       [ 1., -1.,  1.]])]
self.inputs [1 0 1]
nn.weightedInputs [array([1., 2.]), array([2., 2.])]
nn.outputs [array([1., 2.]), array([2., 2.])]
type(self.targets) <class 'list'>
nn.error2 5.0


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [34]:
def activationFunction(x):
    onesArray = np.ones(len(x))
    return 1./(1+np.exp(-x)) 

numberOfNodes,  activationFunction, learningRate, targets, inputs, errorTolerance, maxIterations = \
[2,2,2], activationFunction, 0.1, np.array([1, 0]), np.array([0, 1]), 1e-6, 100000
inputs = np.array((1, 0, 1))

nn3=NN(numberOfNodes, activationFunction, learningRate, targets, inputs, errorTolerance, maxIterations, test=True)
nn3.run()

self.inputs [1 0 1]
type(self.targets) <class 'list'>


TypeError: object of type 'numpy.float64' has no len()